## Legacy reporter 


A simple reporter that only saves conformations in a "legacy" format - using polymerutils.save 


In [1]:
import polychrom
import numpy as np 
import warnings
import h5py 
import glob
from polychrom.simulation import Simulation
import polychrom.starting_conformations
import polychrom.forces, polychrom.forcekits
import simtk.openmm 
import os 
import shutil
import polychrom.polymerutils

### Loading reporter and utils from a hdf5_format module 


In [2]:
from polychrom.legacy.legacy_format import LegacyReporter

### Making a simulation and passing a reporter 

In [3]:
%rm  test_legacy/*
data = polychrom.starting_conformations.grow_cubic(10000,30)

"""
Here we created a hdf5Reporter attached to a foler test, and we are saving 5 blocks per file 
(you should probalby use 50 here or 100. 5 is just for a showcase)
"""
reporter = LegacyReporter(folder="test_legacy")


"""
Passing a reporter to the simulation object - many reporters are possible, and more will be added in a future
"""
sim = Simulation(N=10000, error_tol=0.001, collision_rate=0.01, integrator ="variableLangevin", platform="CPU", 
                reporters=[reporter])
sim.setData(data)
sim.addForce(polychrom.forcekits.polymerChains(sim))
sim._applyForces()
sim.addForce(polychrom.forces.sphericalConfinement(sim, density=0.1))


for i in range(19):        
    """
    Here we pass two extra records: a string and an array-like object.
    First becomes an attr, and second becomes an HDF5 dataset
    """
    sim.doBlock(10, saveExtras={"eggs": "I don't eat green eggs and ham!!!", "spam":[1,2,3]})

"""
Here we are not forgetting to dump the last set of blocks that the reporter has. 
We have to do it at the end of every simulation. 

I tried adding it to the destructor to make it automatic,
but some weird interactions with garbage collection made it not very useable. 
"""
reporter.dump_data()




INFO:root:adding force HarmonicBonds 0
INFO:root:adding force Angle 1
INFO:root:adding force PolynomialRepulsive 2
INFO:root:Particles loaded. Potential energy is 0.051229


Exclude neighbouring chain particles from PolynomialRepulsive
Number of exceptions: 9999


INFO:root:block    1 pos[1]=[13.9 13.8 14.0] dr=0.21 t=0.9ps kin=8.23 pot=3.97 Rg=10.964 dt=24.1fs dx=15.41pm 
INFO:root:block    2 pos[1]=[13.9 13.7 14.1] dr=0.17 t=1.2ps kin=3.79 pot=8.95 Rg=10.965 dt=20.5fs dx=8.93pm 
INFO:root:block    3 pos[1]=[13.9 13.7 14.1] dr=0.09 t=1.4ps kin=7.77 pot=5.02 Rg=10.967 dt=24.5fs dx=15.28pm 
INFO:root:block    4 pos[1]=[13.8 13.6 14.1] dr=0.14 t=1.6ps kin=6.31 pot=6.49 Rg=10.971 dt=22.2fs dx=12.46pm 
INFO:root:block    5 pos[1]=[13.8 13.6 14.1] dr=0.11 t=1.8ps kin=6.75 pot=6.02 Rg=10.977 dt=21.9fs dx=12.71pm 
INFO:root:block    6 pos[1]=[13.8 13.6 14.2] dr=0.12 t=2.0ps kin=6.88 pot=5.87 Rg=10.982 dt=21.9fs dx=12.82pm 
INFO:root:block    7 pos[1]=[13.8 13.5 14.2] dr=0.12 t=2.3ps kin=6.86 pot=5.86 Rg=10.987 dt=21.9fs dx=12.81pm 
INFO:root:block    8 pos[1]=[13.8 13.5 14.2] dr=0.12 t=2.5ps kin=7.16 pot=5.54 Rg=10.993 dt=21.9fs dx=13.09pm 
INFO:root:block    9 pos[1]=[13.9 13.5 14.2] dr=0.12 t=2.7ps kin=6.82 pot=5.85 Rg=10.999 dt=21.9fs dx=12.77pm 
IN

### This is a list of files created in the trajectory folder

In [4]:
!ls -la test_legacy

total 3808
drwxrwxr-x 2 magus magus   4096 Sep  2 10:57 .
drwxrwxr-x 5 magus magus   4096 Sep  2 10:56 ..
-rw-rw-r-- 1 magus magus 202158 Sep  2 10:57 block0.dat
-rw-rw-r-- 1 magus magus 202129 Sep  2 10:57 block10.dat
-rw-rw-r-- 1 magus magus 202144 Sep  2 10:57 block11.dat
-rw-rw-r-- 1 magus magus 202140 Sep  2 10:57 block12.dat
-rw-rw-r-- 1 magus magus 202144 Sep  2 10:57 block13.dat
-rw-rw-r-- 1 magus magus 202138 Sep  2 10:57 block14.dat
-rw-rw-r-- 1 magus magus 202143 Sep  2 10:57 block15.dat
-rw-rw-r-- 1 magus magus 202131 Sep  2 10:57 block16.dat
-rw-rw-r-- 1 magus magus 202137 Sep  2 10:57 block17.dat
-rw-rw-r-- 1 magus magus 202141 Sep  2 10:57 block18.dat
-rw-rw-r-- 1 magus magus 202145 Sep  2 10:57 block1.dat
-rw-rw-r-- 1 magus magus 202153 Sep  2 10:57 block2.dat
-rw-rw-r-- 1 magus magus 202146 Sep  2 10:57 block3.dat
-rw-rw-r-- 1 magus magus 202156 Sep  2 10:57 block4.dat
-rw-rw-r-- 1 magus magus 202160 Sep  2 10:57 block5.dat
-rw-rw-r-- 1 magus magus 202154 Sep  2 10:57 

In [5]:
polychrom.polymerutils.load("test_legacy/block5.dat")

array([[13.83 , 13.575, 14.195],
       [14.047, 14.09 , 14.929],
       [14.091, 13.117, 14.986],
       ...,
       [16.301, 13.966, 15.068],
       [15.428, 14.105, 14.458],
       [14.834, 14.411, 13.615]])